In [1]:
import pandas as pd

In [2]:
game = pd.read_csv('../data/playbyplay/2018/0021700720.csv', dtype={'GAME_ID':str,'SCOREMARGIN':str})

In [3]:
game.head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
0,0021700720,2,12,0,1,8:11 PM,12:00,NaN,NaN,NaN,NaN,NaN
1,0021700720,4,10,0,1,8:11 PM,12:00,Jump Ball Davis vs. Capela: Tip to Anderson,NaN,NaN,NaN,NaN
2,0021700720,7,5,40,1,8:11 PM,11:50,NaN,NaN,Capela Out of Bounds Lost Ball Turnover (P1.T1),NaN,NaN
3,0021700720,8,1,58,1,8:11 PM,11:37,Davis 6' Turnaround Hook Shot (2 PTS) (Holiday...,NaN,NaN,0 - 2,2
4,0021700720,10,2,5,1,8:12 PM,11:12,NaN,NaN,MISS Capela 2' Layup,NaN,NaN


In [4]:
g = game[(game.SCORE.notnull()) & (game.HOMEDESCRIPTION.notnull() | game.VISITORDESCRIPTION.notnull())]

In [5]:
g.head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
3,0021700720,8,1,58,1,8:11 PM,11:37,Davis 6' Turnaround Hook Shot (2 PTS) (Holiday...,NaN,NaN,0 - 2,2
7,0021700720,13,1,1,1,8:12 PM,10:54,Holiday 20' Jump Shot (2 PTS) (Moore 1 AST),NaN,NaN,0 - 4,4
12,0021700720,20,1,6,1,8:13 PM,10:14,NaN,NaN,Harden 1' Driving Layup (2 PTS),2 - 4,2
13,0021700720,21,1,108,1,8:13 PM,10:01,Moore 1' Cutting Dunk Shot (2 PTS) (Cousins 1 ...,NaN,NaN,2 - 6,4
14,0021700720,23,1,108,1,8:13 PM,9:48,NaN,NaN,Capela Cutting Dunk Shot (2 PTS) (Paul 1 AST),4 - 6,2


In [6]:
g.to_csv('tmp.csv',)

In [7]:
def get_scoring_team(row):
    if row['HOMEDESCRIPTION'] == '':
        return 'HOME'
    if row['VISITORDESCRIPTION'] == '':
        return 'AWAY'
    return 'NULL'

In [8]:
def calculate_time(row):
    quarter = int(row['PERIOD'])
    minutes = int(row['PCTIMESTRING'].split(':')[0])
    seconds = int(row['PCTIMESTRING'].split(':')[1])
    return 2880 - (720)*(quarter - 1) - (11 - minutes)*(60) - (60 - seconds)

In [9]:
def process(df):
    g = df[
        (df.SCORE.notnull()) &
        (df.HOMEDESCRIPTION.notnull() | df.VISITORDESCRIPTION.notnull()) &
        (df.PERIOD <= 4)
    ].sort_values('EVENTNUM')
    
    g['TIME'] = g.apply(calculate_time, axis=1)
    
    g['AWAY'] = g['SCORE'].str.split(' - ').apply(lambda x: x[1])
    g['HOME'] = g['SCORE'].str.split(' - ').apply(lambda x: x[0])
    
    g['HOMEDESCRIPTION'].fillna('', inplace=True)
    g['VISITORDESCRIPTION'].fillna('', inplace=True)
    
    g['SCORINGTEAM'] = g.apply(get_scoring_team, axis=1)
    
    g['HOMEMARGIN'] = g['SCOREMARGIN'].apply(lambda x: int(x.replace('TIE','0')))
    
    return g#[['GAME_ID','EVENTNUM','PERIOD','TIME','AWAY','HOME','SCORINGTEAM','HOMEMARGIN','SCORE']]

In [10]:
pro = process(game)

In [11]:
pro.head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,TIME,AWAY,HOME,SCORINGTEAM,HOMEMARGIN
3,0021700720,8,1,58,1,8:11 PM,11:37,Davis 6' Turnaround Hook Shot (2 PTS) (Holiday...,NaN,,0 - 2,2,2857,2,0,AWAY,2
7,0021700720,13,1,1,1,8:12 PM,10:54,Holiday 20' Jump Shot (2 PTS) (Moore 1 AST),NaN,,0 - 4,4,2814,4,0,AWAY,4
12,0021700720,20,1,6,1,8:13 PM,10:14,,NaN,Harden 1' Driving Layup (2 PTS),2 - 4,2,2774,4,2,HOME,2
13,0021700720,21,1,108,1,8:13 PM,10:01,Moore 1' Cutting Dunk Shot (2 PTS) (Cousins 1 ...,NaN,,2 - 6,4,2761,6,2,AWAY,4
14,0021700720,23,1,108,1,8:13 PM,9:48,,NaN,Capela Cutting Dunk Shot (2 PTS) (Paul 1 AST),4 - 6,2,2748,6,4,HOME,2


In [12]:
pd.melt(
    pro,
    id_vars=['GAME_ID','EVENTNUM','PERIOD','TIME','SCORINGTEAM','HOMEMARGIN','SCORE'],
    var_name='SIDE',
    value_name='POINTS',
).sort_values('EVENTNUM')\
.query('SCORINGTEAM == SIDE')

,GAME_ID,EVENTNUM,PERIOD,TIME,SCORINGTEAM,HOMEMARGIN,SCORE,SIDE,POINTS
1000,0021700720,8,1,2857,AWAY,2,0 - 2,AWAY,2
1001,0021700720,13,1,2814,AWAY,4,0 - 4,AWAY,4
1127,0021700720,20,1,2774,HOME,2,2 - 4,HOME,2
1003,0021700720,21,1,2761,AWAY,4,2 - 6,AWAY,6
1129,0021700720,23,1,2748,HOME,2,4 - 6,HOME,4
1005,0021700720,36,1,2720,AWAY,3,4 - 7,AWAY,7
1006,0021700720,37,1,2720,AWAY,4,4 - 8,AWAY,8
1132,0021700720,38,1,2711,HOME,1,7 - 8,HOME,7
1008,0021700720,44,1,2684,AWAY,3,7 - 10,AWAY,10
1009,0021700720,68,1,2578,AWAY,5,7 - 12,AWAY,12
